In [1]:
import subprocess
import sys

packages = ['pandas', 'numpy', 'scikit-learn', 'matplotlib', 'seaborn', 'plotly',
            'folium', 'geopy', 'kaleido']

for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("✓ All packages installed")

✓ All packages installed


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium
from folium import plugins
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
import warnings
import json
from io import StringIO

warnings.filterwarnings('ignore')
np.random.seed(42)

print("✓ All libraries imported successfully")


✓ All libraries imported successfully


In [9]:
ESI_WEIGHTS = {
    'pm25': 0.30,
    'heat': 0.15,
    'pop_density': 0.20,
    'green_deficit': 0.20,
    'vulnerable_pop': 0.10,
    'waste': 0.05,
}

N_CLUSTERS = 3
CLUSTERING_SEED = 42

STRESS_COLORS = {
    'Low Stress': '#2ecc71',
    'Medium Stress': '#f39c12',
    'High Stress': '#e74c3c',
}

# Resource allocation budgets
RESOURCE_BUDGET = {
    'High Stress': {
        'total_budget_per_ward': 5000000,
        'green_infrastructure': 0.40,
        'pollution_control': 0.35,
        'health_monitoring': 0.15,
        'research': 0.10,
    },
    'Medium Stress': {
        'total_budget_per_ward': 2000000,
        'green_infrastructure': 0.35,
        'pollution_control': 0.30,
        'health_monitoring': 0.20,
        'research': 0.15,
    },
    'Low Stress': {
        'total_budget_per_ward': 500000,
        'green_infrastructure': 0.30,
        'pollution_control': 0.20,
        'health_monitoring': 0.30,
        'research': 0.20,
    }
}

print(f"✓ Configuration loaded")

✓ Configuration loaded


In [11]:
#Pune District Boundaries (approximately):
# North: 19.2° N (Ambegaon, Bhhor, Manchar)
# South: 17.5° N (Jejuri, Saswad)
# East: 75.0° E (Pashan, Paud)
# West: 73.5° E (Shirur, Narayangaon)
# City Center: 18.5204° N, 73.8567° E

pune_csv_content = """ward,pm25,heat,pop_density,green_cover,lat,lon,area_name,population,distance_from_city_center_km
Ward_1,84.93,31.46,7593.52,0.0431,18.520,73.856,Pune City Center,58000,0.5
Ward_2,72.23,27.46,14554.00,0.1158,18.555,73.880,Viman Nagar,110000,4.2
Ward_3,87.95,26.88,18500.00,0.1157,18.515,73.910,Kharadi,140000,6.8
Ward_4,105.46,27.50,9033.12,0.1158,18.545,73.815,Wakad,69000,5.5
Ward_5,70.32,30.33,10517.96,0.2200,18.575,73.845,Aundh,80000,4.8
Ward_6,70.32,29.16,12848.78,0.1185,18.495,73.825,Hadapsar,98000,6.2
Ward_7,106.58,27.44,10737.84,0.1468,18.505,73.780,Pune City South,82000,5.0
Ward_8,90.35,29.53,7072.68,0.1377,18.465,73.875,Lohegaon,54000,7.5
Ward_9,65.61,28.69,12739.97,0.1226,18.615,73.815,Yerawada,97000,8.2
Ward_10,85.85,30.44,8781.94,0.0742,18.435,73.895,Rajgurunagar,67000,9.8
Ward_11,65.73,27.10,14157.57,0.1279,18.625,73.755,Chakan,108000,18.5
Ward_12,65.69,27.84,9282.02,0.0514,18.405,73.920,Bhor,71000,22.0
Ward_13,79.84,27.72,17924.77,0.0782,18.355,73.780,Jejuri,137000,28.0
Ward_14,45.00,26.00,9758.61,0.0657,18.675,73.920,Narayangaon,74000,25.5
Ward_15,45.00,29.09,11372.78,0.0941,18.705,73.850,Shirur,87000,30.0
Ward_16,63.75,29.02,15347.31,0.2057,18.555,73.720,Indapur,117000,32.5
Ward_17,54.74,28.51,8191.97,0.0300,18.595,73.695,Mandawa,63000,28.0
Ward_18,81.28,28.03,13296.11,0.1243,18.645,73.890,Talegaon,102000,24.0
Ward_19,56.84,26.00,17074.99,0.0300,18.715,73.920,Ambegaon,130000,35.0
Ward_20,46.75,27.66,6873.81,0.0664,18.475,73.750,Manchar,53000,20.5
Ward_21,52.22,27.81,4053.07,0.3800,18.325,73.870,Saswad,31000,32.0
Ward_22,38.40,26.90,5928.00,0.1556,18.285,73.920,Purandar,45000,35.5
Ward_23,37.06,28.18,5446.48,0.2348,18.345,73.810,Daund,42000,40.0
Ward_24,67.19,29.31,4354.34,0.2805,18.485,73.910,Pimpri,33000,6.5
Ward_25,75.34,32.00,2806.41,0.3800,18.335,73.750,Morgaon,21000,45.0
Ward_26,53.92,28.85,1470.31,0.1551,18.555,73.620,Bhhor,11000,35.0
Ward_27,70.05,29.02,3606.97,0.3631,18.285,73.770,Pen,28000,48.0
Ward_28,60.42,28.35,6212.80,0.2708,18.415,73.700,Ozar,48000,42.0
Ward_29,45.32,26.00,4928.19,0.1915,18.645,73.620,Velha,38000,50.0
Ward_30,60.42,28.45,2008.52,0.3070,18.725,73.780,Mahabaleshwar,15000,55.0
Ward_31,78.07,28.62,4846.36,0.2859,18.575,73.920,Murbad,37000,45.0
Ward_32,54.46,32.00,5270.63,0.2220,18.365,73.650,Kanhe,40000,48.0
Ward_33,78.47,28.12,2732.29,0.2756,18.425,73.820,Katraj,21000,12.0
Ward_34,35.00,29.10,4807.45,0.2392,18.465,73.945,Wagholi,37000,10.0
Ward_35,67.33,28.43,4616.42,0.2791,18.525,73.945,Kalyani Nagar,35000,3.5
Ward_36,56.31,26.16,2214.06,0.3230,18.395,73.750,Sinhagad,17000,18.0
Ward_37,50.51,30.79,5215.57,0.3800,18.665,73.750,Ambegaon East,40000,32.0
Ward_38,56.38,30.00,5621.57,0.1710,18.605,73.620,Kondhwa,43000,20.0
Ward_39,35.00,30.08,6666.10,0.3800,18.445,73.655,Undri,51000,22.0
Ward_40,51.70,26.68,6607.60,0.1200,18.715,73.800,Lonand,50000,38.0
Ward_41,60.36,31.31,1744.66,0.2579,18.595,73.580,Velha East,13000,52.0"""

pune_df = pd.read_csv(StringIO(pune_csv_content))

print(f"✓ Pune district data loaded: {pune_df.shape}")
print(f"  Latitude range: {pune_df['lat'].min():.4f}° to {pune_df['lat'].max():.4f}°N")
print(f"  Longitude range: {pune_df['lon'].min():.4f}° to {pune_df['lon'].max():.4f}°E")
print(f"  Distance from city center: {pune_df['distance_from_city_center_km'].min():.1f}km to {pune_df['distance_from_city_center_km'].max():.1f}km")
print("\nSample wards across district:")
print(pune_df[['ward', 'area_name', 'lat', 'lon', 'pm25', 'distance_from_city_center_km']].head(10))


✓ Pune district data loaded: (41, 10)
  Latitude range: 18.2850° to 18.7250°N
  Longitude range: 73.5800° to 73.9450°E
  Distance from city center: 0.5km to 55.0km

Sample wards across district:
      ward         area_name     lat     lon    pm25  \
0   Ward_1  Pune City Center  18.520  73.856   84.93   
1   Ward_2       Viman Nagar  18.555  73.880   72.23   
2   Ward_3           Kharadi  18.515  73.910   87.95   
3   Ward_4             Wakad  18.545  73.815  105.46   
4   Ward_5             Aundh  18.575  73.845   70.32   
5   Ward_6          Hadapsar  18.495  73.825   70.32   
6   Ward_7   Pune City South  18.505  73.780  106.58   
7   Ward_8          Lohegaon  18.465  73.875   90.35   
8   Ward_9          Yerawada  18.615  73.815   65.61   
9  Ward_10      Rajgurunagar  18.435  73.895   85.85   

   distance_from_city_center_km  
0                           0.5  
1                           4.2  
2                           6.8  
3                           5.5  
4                 

In [12]:
def preprocess_city_data(df, city_name='pune'):
    """Complete preprocessing pipeline"""
    print(f"\n{'='*60}")
    print(f"Processing {city_name.upper()}")
    print(f"{'='*60}")

    df = df.copy()

    if df.isnull().any().any():
        print(f"⚠️  Missing values detected, filling with median...")
        df = df.fillna(df.median())

    scaler = MinMaxScaler()
    feature_cols = ['pm25', 'heat', 'pop_density', 'green_cover']
    df[['pm25_norm', 'heat_norm', 'pop_density_norm', 'green_cover_norm']] = scaler.fit_transform(df[feature_cols])

    df['green_deficit'] = 1 - df['green_cover_norm']

    np.random.seed(42 + hash(city_name) % 1000)
    df['vulnerable_pop'] = np.clip(
        np.random.normal(25, 8, len(df)) + (df['pop_density_norm'] * 10),
        5, 45
    )
    df['waste'] = np.clip(
        np.random.normal(15, 4, len(df)) + (df['pop_density_norm'] * 10),
        5, 35
    )

    df['vulnerable_pop_norm'] = (df['vulnerable_pop'] - df['vulnerable_pop'].min()) / (df['vulnerable_pop'].max() - df['vulnerable_pop'].min())
    df['waste_norm'] = (df['waste'] - df['waste'].min()) / (df['waste'].max() - df['waste'].min())

    print(f"✓ Preprocessing complete")

    return df

pune_processed = preprocess_city_data(pune_df, 'pune')



Processing PUNE
✓ Preprocessing complete


In [13]:
def calculate_esi(df, weights=None):
    """Calculate Environmental Stress Index"""
    if weights is None:
        weights = ESI_WEIGHTS

    df = df.copy()

    feature_mapping = {
        'pm25': 'pm25_norm',
        'heat': 'heat_norm',
        'pop_density': 'pop_density_norm',
        'green_deficit': 'green_deficit',
        'vulnerable_pop': 'vulnerable_pop_norm',
        'waste': 'waste_norm',
    }

    esi = np.zeros(len(df))
    for feature, weight in weights.items():
        if feature in feature_mapping:
            col_name = feature_mapping[feature]
            esi += weight * df[col_name]

    df['esi'] = esi
    return df

def cluster_regions(df, n_clusters=3, seed=42):
    """K-Means clustering"""
    X = df[['pm25_norm', 'heat_norm', 'pop_density_norm', 'green_deficit',
             'vulnerable_pop_norm', 'waste_norm']].values

    kmeans = KMeans(n_clusters=n_clusters, random_state=seed, n_init=10)
    df['cluster_id'] = kmeans.fit_predict(X)

    cluster_esi = df.groupby('cluster_id')['esi'].mean().sort_values()
    cluster_map = {
        cluster_esi.index[0]: 'Low Stress',
        cluster_esi.index[1]: 'Medium Stress',
        cluster_esi.index[2]: 'High Stress',
    }

    df['stress_zone'] = df['cluster_id'].map(cluster_map)

    silhouette = silhouette_score(X, df['cluster_id'])
    davies_bouldin = davies_bouldin_score(X, df['cluster_id'])

    print(f"✓ Clustering complete (k={n_clusters})")
    print(f"  Silhouette Score: {silhouette:.4f}")
    print(f"  Davies-Bouldin Index: {davies_bouldin:.4f}")

    print(f"\n  Cluster Distribution:")
    for zone in ['Low Stress', 'Medium Stress', 'High Stress']:
        count = len(df[df['stress_zone'] == zone])
        pct = (count / len(df)) * 100
        mean_esi = df[df['stress_zone'] == zone]['esi'].mean()
        print(f"    {zone:20s}: {count:2d} regions ({pct:5.1f}%) - Mean ESI: {mean_esi:.3f}")

    return df, kmeans

pune_processed = calculate_esi(pune_processed)
pune_processed, pune_kmeans = cluster_regions(pune_processed)


✓ Clustering complete (k=3)
  Silhouette Score: 0.2344
  Davies-Bouldin Index: 1.4418

  Cluster Distribution:
    Low Stress          : 19 regions ( 46.3%) - Mean ESI: 0.349
    Medium Stress       : 10 regions ( 24.4%) - Mean ESI: 0.446
    High Stress         : 12 regions ( 29.3%) - Mean ESI: 0.621


In [14]:
def calculate_resource_allocation(df, budget_config=None):
    """Calculate resource allocation for each ward"""
    if budget_config is None:
        budget_config = RESOURCE_BUDGET

    df = df.copy()

    df['total_budget'] = 0
    df['green_infrastructure_budget'] = 0
    df['pollution_control_budget'] = 0
    df['health_monitoring_budget'] = 0
    df['research_budget'] = 0

    for idx, row in df.iterrows():
        stress_zone = row['stress_zone']
        budget_structure = budget_config[stress_zone]

        base_budget = budget_structure['total_budget_per_ward']

        # Adjust based on population density
        population_factor = min(row['pop_density'] / 15000, 1.5)

        # Adjust based on ESI severity
        if stress_zone == 'High Stress':
            esi_factor = 1.0 + (row['esi'] - 0.65) * 2
        elif stress_zone == 'Medium Stress':
            esi_factor = 1.0 + (row['esi'] - 0.45) * 1.5
        else:
            esi_factor = 1.0

        total_budget = base_budget * population_factor * esi_factor

        df.loc[idx, 'total_budget'] = total_budget
        df.loc[idx, 'green_infrastructure_budget'] = total_budget * budget_structure['green_infrastructure']
        df.loc[idx, 'pollution_control_budget'] = total_budget * budget_structure['pollution_control']
        df.loc[idx, 'health_monitoring_budget'] = total_budget * budget_structure['health_monitoring']
        df.loc[idx, 'research_budget'] = total_budget * budget_structure['research']

    return df

pune_processed = calculate_resource_allocation(pune_processed)

print(f"\n{'='*60}")
print("RESOURCE ALLOCATION SUMMARY")
print(f"{'='*60}")

for zone in ['High Stress', 'Medium Stress', 'Low Stress']:
    zone_data = pune_processed[pune_processed['stress_zone'] == zone]
    total_budget = zone_data['total_budget'].sum()
    avg_budget = zone_data['total_budget'].mean()
    wards_count = len(zone_data)

    print(f"\n{zone}:")
    print(f"  Number of wards: {wards_count}")
    print(f"  Total budget required: ₹{total_budget/10000000:.2f} Crores")
    print(f"  Average budget per ward: ₹{avg_budget/100000:.2f} Lakhs")

total_city_budget = pune_processed['total_budget'].sum()
print(f"\n{'='*60}")
print(f"TOTAL BUDGET FOR ENTIRE PUNE DISTRICT: ₹{total_city_budget/10000000:.2f} Crores")
print(f"{'='*60}")



RESOURCE ALLOCATION SUMMARY

High Stress:
  Number of wards: 12
  Total budget required: ₹5.09 Crores
  Average budget per ward: ₹42.43 Lakhs

Medium Stress:
  Number of wards: 10
  Total budget required: ₹1.09 Crores
  Average budget per ward: ₹10.90 Lakhs

Low Stress:
  Number of wards: 19
  Total budget required: ₹0.29 Crores
  Average budget per ward: ₹1.51 Lakhs

TOTAL BUDGET FOR ENTIRE PUNE DISTRICT: ₹6.47 Crores


In [15]:
def create_district_stress_map(df, city_name='Pune District', center_lat=18.52, center_lon=73.85):
    """
    Create realistic Folium map showing stress zones across Pune district
    """
    print(f"\n✓ Creating realistic Pune district map...")

    # Create base map (zoomed to show entire district)
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=10,  # Wider zoom to see entire district
        tiles='OpenStreetMap'
    )

    # Add district boundary indicator (approximate)
    district_bounds = [
        [(19.2, 73.5), (19.2, 75.0), (17.5, 75.0), (17.5, 73.5), (19.2, 73.5)]
    ]

    folium.PolyLine(
        district_bounds[0],
        color='black',
        weight=2,
        opacity=0.3,
        popup='Pune District Boundary (Approximate)'
    ).add_to(m)

    # Add markers for each ward
    for idx, row in df.iterrows():
        stress_zone = row['stress_zone']
        color = STRESS_COLORS[stress_zone]

        # Determine marker size based on ESI
        radius = 8 + (row['esi'] * 20)

        # Create detailed popup
        popup_text = f"""
        <div style="font-family: Arial; width: 350px;">
        <h4 style="margin: 0 0 10px 0;">{row['ward']} - {row['area_name']}</h4>
        <hr style="margin: 5px 0;">
        <b>Stress Assessment:</b><br>
        Stress Zone: <span style="color: {color}; font-weight: bold;">{stress_zone}</span><br>
        ESI Score: <b>{row['esi']:.3f}</b> (0-1 scale)<br>
        <hr style="margin: 5px 0;">

        <b>Environmental Indicators:</b><br>
        PM2.5: <b>{row['pm25']:.1f}</b> µg/m³ (WHO limit: 35)<br>
        Temperature: <b>{row['heat']:.1f}</b>°C<br>
        Population Density: <b>{row['pop_density']:.0f}</b> persons/km²<br>
        Green Cover: <b>{row['green_cover']:.1%}</b><br>
        Vulnerable Population: <b>{row['vulnerable_pop']:.1f}%</b><br>
        <hr style="margin: 5px 0;">

        <b>Geographic Location:</b><br>
        Area: <b>{row['area_name']}</b><br>
        Population: <b>{row['population']:,}</b><br>
        Distance from City Center: <b>{row['distance_from_city_center_km']:.1f} km</b><br>
        Coordinates: ({row['lat']:.4f}°N, {row['lon']:.4f}°E)<br>
        <hr style="margin: 5px 0;">

        <b>Budget Allocation (3 Years):</b><br>
        <span style="background: #e8f5e9; padding: 8px; display: block; margin: 5px 0; border-radius: 3px;">
        Total: <b>₹{row['total_budget']/100000:.2f} Lakhs</b><br>
        <span style="font-size: 0.9em;">
        • Green Infrastructure: ₹{row['green_infrastructure_budget']/100000:.2f}L<br>
        • Pollution Control: ₹{row['pollution_control_budget']/100000:.2f}L<br>
        • Health Monitoring: ₹{row['health_monitoring_budget']/100000:.2f}L<br>
        • Research: ₹{row['research_budget']/100000:.2f}L
        </span>
        </span>
        </div>
        """

        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=radius,
            popup=folium.Popup(popup_text, max_width=400),
            tooltip=f"{row['ward']} ({stress_zone}, ESI: {row['esi']:.3f})",
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.8,
            weight=2,
            opacity=0.9
        ).add_to(m)

    # Add city center marker
    m.add_child(folium.Marker(
        location=[18.5204, 73.8567],
        popup='Pune City Center',
        icon=folium.Icon(color='blue', icon='info-sign')
    ))

    # Add comprehensive legend
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; right: 50px; width: 320px; height: auto;
                background-color: white; border:3px solid #333; z-index:9999;
                font-size:13px; padding: 15px; border-radius: 8px; box-shadow: 0 0 15px rgba(0,0,0,0.2);">

    <p style="margin: 0; font-weight: bold; border-bottom: 3px solid #333; padding-bottom: 8px; font-size: 15px;">
    🗺️ PUNE DISTRICT - ENVIRONMENTAL STRESS ZONES</p>

    <p style="margin: 10px 0 5px 0; font-weight: bold; font-size: 12px; color: #666;">STRESS LEVELS:</p>
    <p style="margin: 3px 0;"><i style="background:#e74c3c;width:18px;height:18px;display:inline-block;border-radius:50%;"></i>
    <span style="font-weight: bold;">High Stress</span> - Immediate Intervention (ESI > 0.65)</p>
    <p style="margin: 3px 0;"><i style="background:#f39c12;width:18px;height:18px;display:inline-block;border-radius:50%;"></i>
    <span style="font-weight: bold;">Medium Stress</span> - Preventive Measures (ESI 0.45-0.65)</p>
    <p style="margin: 3px 0;"><i style="background:#2ecc71;width:18px;height:18px;display:inline-block;border-radius:50%;"></i>
    <span style="font-weight: bold;">Low Stress</span> - Maintenance Only (ESI < 0.45)</p>

    <p style="margin: 10px 0 5px 0; font-weight: bold; font-size: 12px; color: #666;">CIRCLE SIZE:</p>
    <p style="margin: 3px 0; font-size: 11px;">Larger circles = Higher ESI (more stressed)</p>
    <p style="margin: 3px 0; font-size: 11px;">Smaller circles = Lower ESI (healthier)</p>

    <p style="margin: 10px 0 5px 0; font-weight: bold; font-size: 12px; color: #666;">INTERACTIONS:</p>
    <p style="margin: 3px 0; font-size: 11px;">Click circles for detailed ward information</p>
    <p style="margin: 3px 0; font-size: 11px;">Hover for quick preview of ward name & ESI</p>

    <p style="margin: 10px 0 5px 0; font-weight: bold; font-size: 12px; color: #666;">KEY INFO PER WARD:</p>
    <p style="margin: 3px 0; font-size: 11px;">
    ✓ Environmental data (PM2.5, heat, green cover)<br>
    ✓ Population & location details<br>
    ✓ 3-year budget allocation<br>
    ✓ Distance from city center
    </p>

    <p style="margin: 10px 0 0 0; padding-top: 8px; border-top: 2px solid #eee; font-size: 10px; color: #999;">
    <b>Note:</b> District map shows 41 wards across Pune district territory</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

print(f"\nCreating interactive Pune district map...")
pune_map = create_district_stress_map(pune_processed)
pune_map.save('pune_district_stress_zones_realistic.html')
print("✓ Map saved as 'pune_district_stress_zones_realistic.html'")

# Display map
pune_map


Creating interactive Pune district map...

✓ Creating realistic Pune district map...
✓ Map saved as 'pune_district_stress_zones_realistic.html'


In [19]:
fig = go.Figure()

for zone, color in [('High Stress', '#e74c3c'), ('Medium Stress', '#f39c12'), ('Low Stress', '#2ecc71')]:
    zone_data = pune_processed[pune_processed['stress_zone'] == zone]

    fig.add_trace(go.Scattergeo(
        lon=zone_data['lon'],
        lat=zone_data['lat'],
        mode='markers+text',
        name=zone,
        text=zone_data['ward'],
        textposition='top center',
        marker=dict(
            size=10 + (zone_data['esi'] * 15),
            color=color,
            opacity=0.8,
            line=dict(width=1, color='white')
        ),
        hovertemplate='<b>%{text}</b><br>Lat: %{lat:.4f}°<br>Lon: %{lon:.4f}°<extra></extra>'
    ))

fig.update_geos(
    scope='asia',
    center=dict(lat=18.52, lon=73.85),
    projection_type='mercator',
    showland=True,
    landcolor='rgb(243, 243, 243)',
    showcoastlines=True,
    coastlinecolor='rgb(204, 204, 204)',
)

fig.update_layout(
    title='Environmental Stress Zones - Pune District Geographic Distribution',
    hovermode='closest',
    height=700,
    margin={"r":0,"t":30,"l":0,"b":0}
)

fig.show()

In [20]:
print(f"\n{'='*100}")
print("DETAILED WARD SUMMARY - STRESS ZONES WITH LOCATIONS")
print(f"{'='*100}")

# Sort by stress zone and ESI
summary_df = pune_processed[[
    'ward', 'area_name', 'distance_from_city_center_km', 'lat', 'lon',
    'pm25', 'pop_density', 'green_cover', 'esi', 'stress_zone', 'total_budget'
]].copy()

summary_df['total_budget'] = summary_df['total_budget'].round(0).astype(int)
summary_df = summary_df.sort_values(['stress_zone', 'esi'], ascending=[False, False])

# Display by stress zone
for zone in ['High Stress', 'Medium Stress', 'Low Stress']:
    zone_data = summary_df[summary_df['stress_zone'] == zone]
    print(f"\n{zone.upper()} ZONES ({len(zone_data)} wards)")
    print(f"{'-'*100}")
    print(zone_data.to_string(index=False))



DETAILED WARD SUMMARY - STRESS ZONES WITH LOCATIONS

HIGH STRESS ZONES (12 wards)
----------------------------------------------------------------------------------------------------
   ward       area_name  distance_from_city_center_km    lat    lon   pm25  pop_density  green_cover      esi stress_zone  total_budget
 Ward_3         Kharadi                           6.8 18.515 73.910  87.95     18500.00       0.1157 0.722652 High Stress       7062710
Ward_13          Jejuri                          28.0 18.355 73.780  79.84     17924.77       0.0782 0.717173 High Stress       6777634
 Ward_7 Pune City South                           5.0 18.505 73.780 106.58     10737.84       0.1468 0.691047 High Stress       3873120
Ward_10    Rajgurunagar                           9.8 18.435 73.895  85.85      8781.94       0.0742 0.675935 High Stress       3079155
 Ward_4           Wakad                           5.5 18.545 73.815 105.46      9033.12       0.1158 0.652012 High Stress       3023156


In [21]:
print(f"\n{'='*100}")
print("GEOGRAPHIC ANALYSIS - STRESS ZONES BY LOCATION")
print(f"{'='*100}")

print(f"\nCITY CORE (0-5 km from city center):")
city_core = pune_processed[pune_processed['distance_from_city_center_km'] <= 5]
print(f"  Wards: {', '.join(city_core['ward'].tolist())}")
print(f"  High Stress: {len(city_core[city_core['stress_zone'] == 'High Stress'])}")
print(f"  Medium Stress: {len(city_core[city_core['stress_zone'] == 'Medium Stress'])}")
print(f"  Low Stress: {len(city_core[city_core['stress_zone'] == 'Low Stress'])}")
print(f"  Average PM2.5: {city_core['pm25'].mean():.1f} µg/m³")
print(f"  Average ESI: {city_core['esi'].mean():.3f}")

print(f"\nSUBURBAN ZONES (5-20 km from city center):")
suburban = pune_processed[(pune_processed['distance_from_city_center_km'] > 5) &
                          (pune_processed['distance_from_city_center_km'] <= 20)]
print(f"  Wards: {', '.join(suburban['ward'].tolist())}")
print(f"  High Stress: {len(suburban[suburban['stress_zone'] == 'High Stress'])}")
print(f"  Medium Stress: {len(suburban[suburban['stress_zone'] == 'Medium Stress'])}")
print(f"  Low Stress: {len(suburban[suburban['stress_zone'] == 'Low Stress'])}")
print(f"  Average PM2.5: {suburban['pm25'].mean():.1f} µg/m³")
print(f"  Average ESI: {suburban['esi'].mean():.3f}")

print(f"\nOUTLYING AREAS (>20 km from city center):")
outlying = pune_processed[pune_processed['distance_from_city_center_km'] > 20]
print(f"  Wards: {', '.join(outlying['ward'].tolist())}")
print(f"  High Stress: {len(outlying[outlying['stress_zone'] == 'High Stress'])}")
print(f"  Medium Stress: {len(outlying[outlying['stress_zone'] == 'Medium Stress'])}")
print(f"  Low Stress: {len(outlying[outlying['stress_zone'] == 'Low Stress'])}")
print(f"  Average PM2.5: {outlying['pm25'].mean():.1f} µg/m³")
print(f"  Average ESI: {outlying['esi'].mean():.3f}")



GEOGRAPHIC ANALYSIS - STRESS ZONES BY LOCATION

CITY CORE (0-5 km from city center):
  Wards: Ward_1, Ward_2, Ward_5, Ward_7, Ward_35
  High Stress: 2
  Medium Stress: 1
  Low Stress: 2
  Average PM2.5: 80.3 µg/m³
  Average ESI: 0.581

SUBURBAN ZONES (5-20 km from city center):
  Wards: Ward_3, Ward_4, Ward_6, Ward_8, Ward_9, Ward_10, Ward_11, Ward_24, Ward_33, Ward_34, Ward_36, Ward_38
  High Stress: 5
  Medium Stress: 1
  Low Stress: 6
  Average PM2.5: 72.1 µg/m³
  Average ESI: 0.484

OUTLYING AREAS (>20 km from city center):
  Wards: Ward_12, Ward_13, Ward_14, Ward_15, Ward_16, Ward_17, Ward_18, Ward_19, Ward_20, Ward_21, Ward_22, Ward_23, Ward_25, Ward_26, Ward_27, Ward_28, Ward_29, Ward_30, Ward_31, Ward_32, Ward_37, Ward_39, Ward_40, Ward_41
  High Stress: 5
  Medium Stress: 8
  Low Stress: 11
  Average PM2.5: 56.8 µg/m³
  Average ESI: 0.410


In [22]:
print(f"\n{'='*100}")
print("RESOURCE ALLOCATION STRATEGY BY GEOGRAPHIC ZONE")
print(f"{'='*100}")

for location, location_df in [('City Core', city_core), ('Suburban', suburban), ('Outlying', outlying)]:
    total_allocation = location_df['total_budget'].sum()
    print(f"\n{location.upper()}:")
    print(f"  Total Budget Required: ₹{total_allocation/10000000:.2f} Crores")
    print(f"  Average per Ward: ₹{location_df['total_budget'].mean()/100000:.2f} Lakhs")
    print(f"  Wards Covered: {len(location_df)}")


RESOURCE ALLOCATION STRATEGY BY GEOGRAPHIC ZONE

CITY CORE:
  Total Budget Required: ₹1.04 Crores
  Average per Ward: ₹20.87 Lakhs
  Wards Covered: 5

SUBURBAN:
  Total Budget Required: ₹2.33 Crores
  Average per Ward: ₹19.44 Lakhs
  Wards Covered: 12

OUTLYING:
  Total Budget Required: ₹3.09 Crores
  Average per Ward: ₹12.88 Lakhs
  Wards Covered: 24


In [23]:
pune_processed['primary_intervention'] = pune_processed.apply(
    lambda row: 'Emergency Greening + Pollution Control' if row['stress_zone'] == 'High Stress'
    else ('Preventive Development' if row['stress_zone'] == 'Medium Stress'
    else 'Maintenance Only'), axis=1
)

results_export = pune_processed[[
    'ward', 'area_name', 'lat', 'lon', 'distance_from_city_center_km',
    'pm25', 'heat', 'pop_density', 'green_cover', 'esi', 'stress_zone',
    'total_budget', 'green_infrastructure_budget', 'pollution_control_budget',
    'health_monitoring_budget', 'research_budget', 'primary_intervention', 'population'
]].copy()

results_export = results_export.sort_values('distance_from_city_center_km')
results_export['total_budget'] = results_export['total_budget'].round(0).astype(int)
results_export['green_infrastructure_budget'] = results_export['green_infrastructure_budget'].round(0).astype(int)
results_export['pollution_control_budget'] = results_export['pollution_control_budget'].round(0).astype(int)
results_export['health_monitoring_budget'] = results_export['health_monitoring_budget'].round(0).astype(int)
results_export['research_budget'] = results_export['research_budget'].round(0).astype(int)

results_export.to_csv('pune_district_environmental_inequality_realistic.csv', index=False)
print("\n✓ Detailed results exported to 'pune_district_environmental_inequality_realistic.csv'")

results_json = results_export.to_dict(orient='records')
with open('pune_district_intervention_plan.json', 'w') as f:
    json.dump(results_json, f, indent=2)
print("✓ Intervention plan exported to 'pune_district_intervention_plan.json'")



✓ Detailed results exported to 'pune_district_environmental_inequality_realistic.csv'
✓ Intervention plan exported to 'pune_district_intervention_plan.json'


In [24]:
print(f"\n{'='*100}")
print("YOUR FILES ARE READY - DOWNLOAD INSTRUCTIONS")
print(f"{'='*100}")

print("""
RUN THIS CODE IN A NEW COLAB CELL TO DOWNLOAD:

from google.colab import files

# Download interactive map
files.download('pune_district_stress_zones_realistic.html')

# Download detailed CSV with all data
files.download('pune_district_environmental_inequality_realistic.csv')

# Download intervention plan (JSON)
files.download('pune_district_intervention_plan.json')

FILES GENERATED:
1. pune_district_stress_zones_realistic.html
   ✓ Interactive Folium map
   ✓ Shows all 41 wards across entire Pune district
   ✓ Color-coded stress zones with proper geographic distribution
   ✓ Click any ward for detailed information
   ✓ Realistic coordinates and spacing (no overlaps)

2. pune_district_environmental_inequality_realistic.csv
   ✓ All wards with environmental data
   ✓ Geographic coordinates and area names
   ✓ Budget allocation for each ward
   ✓ Breakdown by intervention type
   ✓ Distance from city center

3. pune_district_intervention_plan.json
   ✓ Machine-readable format
   ✓ For APIs and software integration
   ✓ All data from CSV in JSON structure
""")

print(f"\n{'='*100}")
print("✓ REALISTIC PUNE DISTRICT MAP GENERATION COMPLETE!")
print(f"{'='*100}")
print(f"\nSummary:")
print(f"- Total Wards Analyzed: {len(pune_processed)}")
print(f"- High Stress Zones: {len(pune_processed[pune_processed['stress_zone'] == 'High Stress'])}")
print(f"- Medium Stress Zones: {len(pune_processed[pune_processed['stress_zone'] == 'Medium Stress'])}")
print(f"- Low Stress Zones: {len(pune_processed[pune_processed['stress_zone'] == 'Low Stress'])}")
print(f"- Geographic Spread: {pune_processed['distance_from_city_center_km'].max():.1f} km radius")
print(f"- Total Budget Required: ₹{pune_processed['total_budget'].sum()/10000000:.2f} Crores")



YOUR FILES ARE READY - DOWNLOAD INSTRUCTIONS

RUN THIS CODE IN A NEW COLAB CELL TO DOWNLOAD:

from google.colab import files

# Download interactive map
files.download('pune_district_stress_zones_realistic.html')

# Download detailed CSV with all data
files.download('pune_district_environmental_inequality_realistic.csv')

# Download intervention plan (JSON)
files.download('pune_district_intervention_plan.json')

FILES GENERATED:
1. pune_district_stress_zones_realistic.html
   ✓ Interactive Folium map
   ✓ Shows all 41 wards across entire Pune district
   ✓ Color-coded stress zones with proper geographic distribution
   ✓ Click any ward for detailed information
   ✓ Realistic coordinates and spacing (no overlaps)

2. pune_district_environmental_inequality_realistic.csv
   ✓ All wards with environmental data
   ✓ Geographic coordinates and area names
   ✓ Budget allocation for each ward
   ✓ Breakdown by intervention type
   ✓ Distance from city center

3. pune_district_intervention_pla

In [25]:

from google.colab import files

# Download interactive map
files.download('pune_district_stress_zones_realistic.html')

# Download detailed CSV with all data
files.download('pune_district_environmental_inequality_realistic.csv')

# Download intervention plan (JSON)
files.download('pune_district_intervention_plan.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>